[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/khanyofficial/inkers1/blob/master/DNST_CIFAR10_AUG.ipynb)

In [0]:
from __future__ import print_function

# https://keras.io/
!pip install -q keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Concatenate, Dense, Dropout, Conv2D, MaxPooling2D, BatchNormalization, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Reshape, Lambda, Conv1D, SeparableConv2D
from keras.optimizers import Adam, SGD, RMSprop, Adadelta
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.merge import concatenate
from keras.utils import np_utils
import tensorflow as tf

In [3]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 48s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [7]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 12)   324         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 12)   48          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 12)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
                              
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")
                              
from google.colab import files
files.download('DNST_model.h5')

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 175s 3ms/step - loss: 1.6671 - acc: 0.3784 - val_loss: 1.6126 - val_acc: 0.4415
Epoch 2/50
50000/50000 [==============================] - 145s 3ms/step - loss: 1.3676 - acc: 0.4986 - val_loss: 1.6373 - val_acc: 0.4734
Epoch 3/50
50000/50000 [==============================] - 146s 3ms/step - loss: 1.1912 - acc: 0.5716 - val_loss: 1.3235 - val_acc: 0.5568
Epoch 4/50
50000/50000 [==============================] - 146s 3ms/step - loss: 1.0774 - acc: 0.6128 - val_loss: 1.5327 - val_acc: 0.5232
Epoch 5/50
50000/50000 [==============================] - 146s 3ms/step - loss: 1.0004 - acc: 0.6397 - val_loss: 1.8537 - val_acc: 0.4911
Epoch 6/50
50000/50000 [==============================] - 127s 3ms/step - loss: 0.9566 - acc: 0.6559 - val_loss: 1.1869 - val_acc: 0.6042
Epoch 7/50
50000/50000 [==============================] - 107s 2ms/step - loss: 0.9131 - acc: 0.6731 - val_loss: 1.0480 - 